<a href="https://colab.research.google.com/github/saurabh2086/whisper_use_case/blob/main/whisper_speech_to_text.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#  1. Installing all the libraries

In [ ]:
# Installing whisper
!pip install git+https://github.com/openai/whisper.git
!sudo apt update && sudo apt install ffmpeg

# Installing Spacy model
import locale
locale.getpreferredencoding = lambda: "UTF-8"
!python -m spacy download en_core_web_md

# Installing gradio
! pip install gradio -q

# Importing libraries

In [ ]:
import whisper
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import spacy
import gradio as gr
import pandas as pd
import datetime
import spacy
from spacy.matcher import Matcher, PhraseMatcher
import time
import pytz

# HSBC product catalogue

In [ ]:
# product catelogue
product_name = ["Stocks and Securities Trading",
                "Unit trust investment",
                "Bonds",
                "Structured Product",
                "Dual Currency Plus",
                "Sustainable Investing",
                "FX Services",
                "Everyday Global Account",
                "Wealth Portfolio Lending",
                "Sapphire Prestige Income II",
                "Variable Annuity",
                "Jade Legacy Universal Life",
                "Emerald Legacy Life III",
                "ValueLife Insurance",
                "Term Protect Advantage",
                "Wealth Builder",
                "Retirement Income II",
                "Savings Protector II",
                "Goal Builder",
                "International Health Insurance",
                "Health Insurance - SmartCare Optimum Enhanced"
                ]
product_description = ["""Start trading stocks and shares with our Stock Trading platform. \
If you are an experienced investor, you'll want to be able to trade stocks and securities freely. 
With our online stock trading platform you can trade stocks or other securities types. 
Take advantage of trading opportunities in the US, Singapore and Hong Kong markets.
Trade instantly via HSBC Singapore app or online banking.
You will have access to Stocks, Exchange Traded Funds (ETF's), Real Estate Investment Trusts (REITs), 
Global Depository Receipts (GDRs) and American Depository Receipts (ADRs)
""",
"""Grow your wealth through investments with our range of professionally managed unit trusts (or funds).
A unit trust fund invests in a diverse portfolio of investments. This helps to diversify your risk better than investing in a single security.
You can buy and sell unit trusts on any dealing day with ease - online or through your Relationship Manager.
Unit trusts let you access a portfolio of investments from as low as SGD1,000 - a more cost efficient option than building the portfolio yourself.
Gain exposure to countries /regions, sectors and investments that are not easily accessible as an individual.
Let the experts work for you - your investments are managed by investment professionals who spend their careers researching and managing investments.
""",
"""By investing in bonds, you may receive steady interest income with a potentially higher yield and return compared to the interest you'd earn on cash savings rate over the same amount of time.
Higher interest potential than cash savings accounts.
Diversify your investment risk.
Potential long-term capital gain.
Earn a regular stream of interest income.
Corporate bonds, supranational bonds, government/quasi-government bonds.
Fixed-rate bonds, floating rate coupon bonds.
Available in multiple currencies: USD, AUD, CAD, CNH, EUR, GBP, NZD, SGD.
""",
"""If you're looking to spread the risk in your investment portfolio, Structured Products could help you do just that.
Customisable according to the investor's market view.
Access underlying asset which may otherwise not be readily accessible.
Potential yield enhancements.
The return on a Structured Product is variable and depend on the performance of the underlying asset
Structured Products are investment products where the return is linked to an underlying asset with pre-defined features (tenor, currency, payout calculation etc. )
Underlying asset can be equities, interest rates,  commodities, mutual funds & foreign currencies among others.
""",
"""Invest in currency pairs as underlying assets
Dual Currency Plus helps you make the most of your foreign currency needs by allowing you to pair two currencies of your choice and making use of foreign exchange fluctuation to enhance your returns.
""",
"""Environmental, Social and Governance (ESG) is a set of criteria that can be applied to how a company operates. 
When we talk about ESG investing, we're referring to the use of these criteria as factors in our investment decisions and in evaluating potential companies to add to your financial portfolio.
""",
"""
Get access to easy FX solutions. Check the latest foreign currency exchange rates in real-time online.
Looking to capitalise on foreign exchange rate fluctuations online
Our foreign exchange (or commonly known as forex) solutions are designed to help no matter if you're interested in investing in foreign currencies, or need to send money abroad.
If you're looking to invest, you should know that forex is the world's most traded investment market.
Whether you're an expert or novice investor, our foreign exchange services can help you make the most out of your money.
""",
"""Unlock rewards easily with 1% cashback and enjoy up to SGD300 bonus cash reward.
Introducing a multi-currency account that meets your transactional needs in up to 11 different currencies.
Send money internationally with the HSBC Global Money Transfers
Unlock more benefits with HSBC Everyday+ Rewards Programme
""",
"""Access extra funds for the short term, without cashing out on your existing investments.
Invest in your future with a loan backed by your assets.
Our Wealth Portfolio Lending loan gives you access to extra funds, and this is secured by your existing eligible HSBC assets acceptable to the bank. 
The Effective Credit Limit granted is dependent on the type of assets pledged and will be subject to the Bank's approval.
A loan can help you easily reach your goals, without having to worry about your cashflow.
Plus, you can use your assets flexibly to increase your liquidity, helping to protect your existing wealth and open up new opportunities.
Whether you're looking to venture into new investments, treat yourself or settle any unexpected expenses, 
you can access extra funds without liquidating your existing investments.
""",
"""Secure your legacy with sustainable wealth
Building a lasting financial legacy for your family goes beyond accumulating wealth. It's about empowering future generations with financial stability at every life stage.
HSBC Life Sapphire Income II is a whole life plan that provides you with regular income to complement your savings so you can enjoy your retirement with peace of mind. 
It gives you and your loved ones the financial edge to sustain your wealth and build a legacy that lasts for generations.
""",
"""HSBC Life Variable Annuity is a capital guaranteed1 retirement solution that is designed to provide you with a stream of guaranteed monthly retirement income.
A one-time investment in US dollars, this plan helps to boost your retirement income by giving you the chance to benefit from market upside performance and protect against downside risk.
Capital guaranteed - HSBC Life Variable Annuity retirement solution keeps your capital safe while you stay invested.
Benefit from market upsides / Protection when market falls - Your Guaranteed Payout Base (GPB) is locked-in based on the higher account value, whenever it increases on every monthiversaryof your policy. It will never decrease once it is locked-in33 view footnote 3.
Receive a stream of guaranteed monthly payouts and potential lump sum at the end of the policy term to fund your retirement.
High liquidity - access your account value any time without charge!
""",
"""Jade Legacy Universal Life Plan and Jade Ultra Legacy Universal Life Plan are designed for individuals or business owners who are seeking life insurance protection and potential wealth accumulation.
Your financial circumstances and goals for the future are unique to you. Customise the solution to meet a wide range of estate-planning needs, thus securing your legacy through the generations and the financial future of your loved ones.
Provide more for your legacy without compromising your retirement lifestyle
Grow your cash value with guaranteed interest rate lock option and exclusive loyalty bonus
Build a stronger foundation for your future generations with multi-generational planning
Preserving the value of the assets you acquired for your loved ones
""",
"""Your legacy is much more than the wealth you leave behind for your family. We are here to help you grow your legacy and ensure your plans for your loved ones are taken care of, and lead them to greater success with financial assurance and support.
Grow your legacy with HSBC Life Emerald Legacy Life III - a single premium whole life plan. So you can rest assured that your wealth is preserved for your family or passed on for many more generations to come.
Coverage for death and terminal illness, up to age 99.
Option to replace the life insured when the need arises, for both corporate and individual policies11 view footnote 1.
5% of the single premium paid will be refunded to you when the life insured quits smoking anytime22 view footnote 2 within the first 4 policy years.
Available in both SGD and USD.
""",
"""At HSBC, we understand you want the assurance of lifetime protection, without the burden of paying premiums in your golden years. Which is why we have designed ValueLife to offer you the freedom to live life to the fullest. 
It's a smarter way to invest in what matters most - your future.
ValueLife is a non-participating whole life insurance plan that lets you reduce your coverage, at either ANB 65 or 80.
When it comes to your later stages in life, most of your life and financial goals would have been achieved. Unlike typical whole-of-life plans, you can reduce your coverage, creating an opportunity for cost savings.
Guaranteed lifetime protection with a limited premium paying term
ValueLife is designed with your future in mind. Pay regular premiums for only the first 15 years and be insured against Death and Terminal Illness up to ANB 99, and Total and Permanent Disability before ANB 65.
Guaranteed refund of premiums
With ValueLife, you’ll receive 100% of the total premiums paid at the end of year 15, if you decide to surrender your policy and if there is no premium loading.
Guaranteed maturity benefit
If no claims have been made up to ANB 99, the reduced Sum Insured, i.e. 70% of the basic Sum Insured, will be paid out in one lump sum as the maturity benefit.
""",
"""With Term Protect Advantage, you have the flexibility to choose your preferred premium and policy term, and receive the surrender value11 view footnote 1, when the coverage is no longer needed.
Affordable protection coverage.Challenging times call for an affordable policy, which protects you against death terminal illness, so you can allocate your money to where you need it most.
Flexible premium and policy term.Take control of your financial future by choosing the term of your coverage and how long you wish to pay for it.
Boost your coverage with Guaranteed Insurability Option.Our financial needs often change with our life stages. With Term Protect Advantage, you can increase your coverage at life key milestones without medical underwriting.
Convertibility benefit.Later in life, you may change to any whole life or endowment plan (that is available to you at that point in time) without medical underwriting.
Unemployment support benefit.You may pay your premium at a later date (up to 365 days) if you become unemployed.
Guaranteed surrender value.When you no longer need the coverage, you can surrender your policy and receive the guaranteed surrender value of your policy (available for only limited pay option).
""",
"""HSBC Life Wealth Builder is a flexible, savings plan that allows you to potentially grow your wealth through long-term accumulation (up to age 12011 view footnote 1), giving the financial assurance you need for you, your child, possibly even your grandchildren.
Flexibility to build a plan to your preference.Choose premium term of Single Premium, 5, 10, or 15 years.
Option to pause your premiums payment without policy lapsing.
Option to jointly buy the policy with your spouse.
Receive both guaranteed and non-guaranteed bonuses.
Access to the cash value of the policy.While the plan builds its cash value, you can still access to your savings whenever you need to fund key milestones.
Policy continuity to ensure the cash value keep growing.Option to appoint your spouse or child as the Secondary Life Insured.
Remaining premiums are waived if you suffer from Total and Permanent Disability (TPD).
""",
"""
HSBC Life Retirement Income II gives you monthly financial support so you can keep the promises you’re most looking forward to
Retire on your own terms. Tailor your policy so you can get closer to fulfilling your retirement aspirations.  Customise your plan with the following options:
Retirement Age at 55, 60, 65, 70. Desired Guaranteed Monthly Payout from S$300.
Flexible premium term options - Single Premium, 5 or 10 years.
Income Payout Period with 10, 15, 20 years or to age 99.
Fund your policy with cash or monies from Supplementary Retirement Scheme account.
Unemployment Support Benefit.n case of Unemployment you have the option to defer premium payments for up to 365 days.
All future premiums will be waived if you or your spouse is diagnosed with Total and Permanent Disability, or in the event of your spouse’s death.
Chronic Illness Care Benefit.Receive additional 100% of the Guaranteed Monthly Payout if diagnosed with Loss of Independence, Idiopathic Parkinson's Disease, Alzheimer's Disease/ Severe Dementia or Major Head Trauma.
Mobility Care Benefit.Receive a single lump sum payout worth 20 times Guaranteed Monthly Payout (up to S$25,000) if diagnosed with Osteoporosis with Fractures or Severe Rheumatoid Arthritis.
""",
"""HSBC Life Savings Protector II provides assurance to your loved ones that they can count on the commitment you make. Seal your commitments with HSBC Life Savings Protector II which will not only protect your savings but provides flexibility to potentially earn higher returns.
Whether it is planning for a comfortable retirement or providing for your children's education, HSBC Life Savings Protector II can help you to seal your goals. This plan is suitable if you wish to have a higher protection coverage to protect your financial goals.
HSBC Life Savings Protector II guarantees your capital11 view footnote 1 at the end of the policy term even during a market downturn. You can also potentially enjoy higher payouts through non-guaranteed reversionary22 view footnote 2 and terminal33 view footnote 3 bonuses on maturity.
HSBC Life Savings Protector II can be tailored to best fit your needs. You can choose premium payment terms of 3 or 5 years to suit your financial situation.
""",
"""Our Goal Builder is a limited premium investment-linked plan that can give you lifelong protection. With over 70 world-class funds to choose from, it also provides high flexibility to grow your wealth.
If you want to grow your wealth and protect yourself and your family at the same time, the HSBC Life Goal Builder could be the right option for you.
Access to over 70 world-class funds
No premium charges, and all basic premiums go towards investing into funds of your choice
Option to re-balance your fund holdings automatically
No product administrative fees44 view footnote 4 from policy year 25 onwards
Top up premiums allowed at any time through both ad-hoc or regular top up arrangement.
Unlimited ad hoc or pre-planned regular withdrawal.Potentially create an income stream by investing in dividend-paying funds.
Lifelong protection for death or terminal illness.Complimentary accidental death coverage up to age 75.
Option to enhance the death benefits up to 10 times of annual premium.No medical examinations required.Available in both SGD and USD.
""",
"""Hospital charges incurred during an in-patient hospital stay are covered, giving you peace of mind throughout your hospital treatment.
Out-patient benefits including cover for general practitioner and specialist consultations, prescribed drugs and dressings and necessary diagnostic tests, and vaccinations up to the limits shown.
Depending on where in the world you reside, travel to or wish to receive treatment, there are 3 geographical areas of cover designed to suit your needs: Asia, Worldwide excluding USA, and Worldwide
""",
"""No limit to number of days of hospital stay.Dread disease recuperation.Cash allowance while hospitalised.Inpatient psychiatric treatment.
Greater coverage, Cashless outpatient visits to our panel general practitioners.
Alternative treatment under Platinum and Gold Plan i.e. consultation and treatment provided by registered chiropractor, podiatrist, dietitian, nutritionist, naturopath, acupuncturist, homeopath, osteopath, TCM and physiotherapist.
Pediatrician consultation benefit (waiver of general practitioner referral letter for insured child below 36 months).
As charged benefits.No co-payment & deductible.Cover up to age 80.Stable portfolio pricing i.e. HSBC Life (Singapore) Pte. Ltd. will not impose capping on your medical conditions or additional loadings on your premium upon renewal as a result of your personal claims experience.
While most health insurance plans cover you for hospitalisation only, SmartCare Optimum Enhanced now offers you optional riders to complement your core plan.
Dental rider – provides coverage for outpatient visit to dentist (20% co-payment applies).
Travel inconvenience rider – enhance your travel benefits for your next vacation or business trips.
With just a call from anywhere in the world, you can stay connected to our 24-hour customer service assistance hotline and get access to assistance for emergency medical evacuation and repatriation services.
With a choie of three comprehensive plans, you have the flexibility of selecting the plan that best suits your needs and budget.
With SmartCare Optimum Enhanced there is no restriction on your choice of hospital. This means that you can choose to receive treatment in a public or a private hospital.
"""
]

#### storing to a dataframe

In [ ]:
product_df = pd.DataFrame(dict(products=product_name, description=product_description))
products = product_df["products"].values.tolist()

# Transcription text example

In [ ]:
text = """Hi, today I met a client at the Revo City branch and we had a discussion for about one hour. 
The name of the client was Mr. XYZ and we talked about various financial aspects of the client. 
Based on our discussion the short term goals of the client were borrow money to fund his housing purchase. He wants to use
his existing funds to borrow loan which can act as a collateral.
He also wants to diversify his current investment by investing in multiple products such as commodities currency and
stocks.
The long term goals of this client to the few
long term plans like my sending his kids to college in the US and buying a few more investment properties as well as 
he wants to invest in bond. He is also concerned about insurance so currently he has medical insurance but he 
wants to take a life insurance as well that will cover him for life. He is looking for a term for maybe around 
99 years or something. The client is happy about  with HSBC that HSBC is in multiple countries so he wants to
fund the EGA account using which he can have his money in various currencies. He also likes the network of HSBC and the 
services we provide. The thing he is not happy about the latency between fund transfers. Sometimes it takes one or 
two days for funds to reflect. He was asking if we have any plans to change that. On the next step I suggest 
that we should plan the insurance part of this client since the other plans are more merely short terms and 
for the long term plans we need to put the right insurance in place, and he requires more FX services in the future"""

# Supporting functions

#### define patterns


In [ ]:
short_term_pattern = [{"LOWER":"short"},{"ORTH":"-","OP":"?"},{"LOWER": "term"}, {"LEMMA": "goal"}]
long_term_pattern = [{"LOWER":"long"},{"ORTH":"-","OP":"?"}, {"LOWER": "term"}, {"LEMMA": "goal"}]
happy_pattern = [{"LOWER": "happy"}, {"LOWER": "about"}]
not_happy_pattern = [{"LOWER": "not"}, {"LOWER": "happy"}, {"LOWER": "about"}]
next_steps_pattern = [{"LOWER": "next"}, {"LEMMA": "step"}]
other_info_pattern = [{"LOWER": "other"}, {"LEMMA": "important"}, {"LEMMA": "information"}]


#### Voice to text transcription function 

In [ ]:
def transcribe(file_name):
  if file_name:
    model = whisper.load_model(name='base')
    result = model.transcribe(file_name)
    result = result["text"]
  else:
    result = None 
  return result

#### Loading spacy model

In [ ]:
# Load the spacy model
nlp = spacy.load('en_core_web_md')
nlp_sm = spacy.load('en_core_web_sm')

#### Find short term goals

In [ ]:
def find_short_term_goals(text):
  doc = nlp(text)
  matcher = Matcher(nlp.vocab)
  matcher.add("short_term_pattern",[short_term_pattern])
  matches = matcher(doc)
  return matches

#### Find Long term goals

In [ ]:
def find_long_term_goals(text):
  doc = nlp(text)
  matcher = Matcher(nlp.vocab)
  matcher.add("long_term_pattern",[long_term_pattern])
  matches = matcher(doc)
  return matches

#### Find happy about

In [ ]:
def find_happy_about(text):
  doc = nlp(text)
  matcher = Matcher(nlp.vocab)
  matcher.add("happy_pattern",[happy_pattern])
  matches = matcher(doc)
  return matches

#### Find not happy about

In [ ]:
def find_not_happy_about(text):
  doc = nlp(text)
  matcher = Matcher(nlp.vocab)
  matcher.add("not_happy_pattern",[not_happy_pattern])
  matches = matcher(doc)
  return matches

#### Find next steps

In [ ]:
def find_next_steps(text):
  doc = nlp(text)
  matcher = Matcher(nlp.vocab)
  matcher.add("next_steps",[next_steps_pattern])
  matches = matcher(doc)
  return matches

#### Find other important information

In [ ]:
def find_other_imp_info(text):
  doc = nlp(text)
  matcher = Matcher(nlp.vocab)
  matcher.add("other_info_pattern",[other_info_pattern])
  matches = matcher(doc)
  return matches

#### find next big number

In [ ]:
def find_stop_index(a,n):
  a = np.sort(a)
  stop_index = a[a > n][0]
  return stop_index

#### find text span

In [ ]:
def find_span(a,match_id,doc):
  if len(match_id) > 0:
    start_id = match_id[0][1]
    end_id = find_stop_index(a,start_id)
    span = doc[start_id:end_id]
    return span.text
  else:
    return None

#### Find all informations in the text

In [ ]:
def find_information(text):
  doc = nlp(text)
  # find short term goal
  short_term = find_short_term_goals(text)
  long_term = find_long_term_goals(text)
  happy = find_happy_about(text)
  not_happy = find_not_happy_about(text)
  next_step = find_next_steps(text)
  other_info = find_other_imp_info(text)

  list_steps = [short_term, long_term,happy,not_happy,next_step,other_info]
  start_indices = np.array([m[0][1] for m in list_steps if len(m)>0])
  start_indices = np.append(start_indices,[len(text)])

  short_term_span = find_span(start_indices,short_term,doc)
  long_term_span = find_span(start_indices,long_term,doc)
  happy_span = find_span(start_indices,happy,doc)
  not_happy_span = find_span(start_indices,not_happy,doc)
  next_step_span = find_span(start_indices,next_step,doc)
  other_info_span = find_span(start_indices,other_info,doc)

  all_info = {"short_term_goals" : short_term_span,
              "long_term_goals" : long_term_span,
              "happy_about" : happy_span,
              "not_happy_about" : not_happy_span,
              "next_steps" : next_step_span,
              "other_imp_info" : other_info_span
              }

  return all_info

#### Find similarity between two texts

In [ ]:
def find_similarity(conversation, description):
  
  # Compute the cosine similarity between the document vectors
  conversation_doc = nlp(conversation)
  description_doc = nlp(description)
  similarity = conversation_doc.similarity(description_doc)

  return similarity

#### Find top three recommendation

In [ ]:
def find_recommendation(prod_desc, conversation, k=3):
  prod_desc['similarity_score'] = (prod_desc['description']
                                   .apply(find_similarity, 
                                          args=(conversation,))
                                   )
  sum_score = prod_desc['similarity_score'].sum()
  prod_desc['norm_sim_score'] =  prod_desc['similarity_score']/sum_score

  top_df = prod_desc.sort_values("norm_sim_score", ascending=False).head(k)

  rec = top_df[["products","similarity_score"]].set_index("products").to_dict("dict")

  return rec["similarity_score"]

  

#### Getting date and time

In [ ]:
# get the current UTC time
utc_time = datetime.datetime.utcnow()
# set the timezone you want to display
timezone = pytz.timezone('Asia/Singapore')
# convert the UTC time to the timezone you want to display
display_time = utc_time.replace(tzinfo=pytz.utc).astimezone(timezone)
# format the display time as a string
display_date_str = display_time.strftime('%Y-%m-%d')
display_time_str = display_time.strftime('%I:%M %p')

# Functions to update components

#### Update transcription text

In [ ]:
def update_transcription_text(audio_file):
  transcription = transcribe(audio_file)
  return transcription

#### Clear audio button

In [ ]:
def clear_audio_button():
  return None

#### Update short-term long-term goals

In [ ]:
def update_short_long_term_goals(text):
  all_info = find_information(text)
  return {short_term_goals: all_info["short_term_goals"],
          long_term_goals: all_info["long_term_goals"]
          }

#### Record New meeting button click

In [ ]:
def record_new_meeting():
  return [None, '']

#### Recommend HSBC product button click

In [ ]:
def recommend_hsbc_product(short_term_goals, long_term_goals):
  if short_term_goals is not None:
    short_rec = find_recommendation(product_df,short_term_goals)
  else: 
    short_rec = None

  if long_term_goals is not None:
    long_rec = find_recommendation(product_df,long_term_goals)
  else: 
    long_rec = None

  recommendations = {short_term_rec: short_rec,
                     long_term_rec: long_rec
                     }
  return recommendations

#### Submit record

In [ ]:
def submit_all_records(df_base1, df_base2,
                       customer_id,customer_name,meeting_date,
                       meeting_time,transcription,
                       short_term_rec,long_term_rec,
                       rm_rec_sort,rm_rec_long):
  update_datetime = datetime.datetime.today()
  cust_info = {"name": customer_name,
               "id": customer_id
               }
  meeting_info = {"date": meeting_date,
                  "time": meeting_time
                  }
  information = find_information(text)

  short_rec = [k['label'] for k in short_term_rec['confidences']]
  long_rec = [k['label'] for k in long_term_rec['confidences']]

  content_based_rec = {"ShortTermRec": short_rec,
                       "LongTermRec": long_rec
                       }
  rm_rec = {"ShortTermRec": rm_rec_sort,
            "LongTermRec": rm_rec_long
            }
  record1 = {
      "update_datetime": [update_datetime],
      "customer_info": [str(cust_info)],
      "meeting_info": [str(meeting_info)],
      "meeting_summary": [str(information)],
      }

  record2 = {
      "update_datetime": [update_datetime],
      "customer_info": [str(cust_info)],
      "content_based_recommendation":[str(content_based_rec)],
      "RM_recommendation":[str(rm_rec)]
      }

  df1 = pd.DataFrame(record1)
  df1 = pd.concat([df_base1,df1],ignore_index=True)

  df2 = pd.DataFrame(record2)
  df2 = pd.concat([df_base2,df2],ignore_index=True)


  return {rm_df: df1,
          df_out: df1,
          rec_df: df2,
          df_rec_out:df2
          }

  

# Design components

In [ ]:
EMPLOYEE_ID = "32467953"
EMPLOYEE_NAME = "Ms Tan"
EMPLOYEE_BRANCH = "Holland Village"
CUSTOMER_ID = "SGHSPE73539362"
CUSTOMER_NAME = "Josué Prieto"
DATE = display_date_str
TIME = display_time_str

INSTRUCTION = """1. Short term goals
2. Long term goals
3. Happy about
4. Not happy about
5. Next steps
6. Other important information"""

# New Design

In [ ]:
with gr.Blocks() as demo:
  gr.Markdown("""# BUDDY - AI ping yu for RMs""")
  gr.Markdown("""### Follow steps 1 to 4. Rinse and repeat. Enjoy !!""")
  gr.Markdown("""Send comments to gunjan.x.bhatt@hsbc.com.sg""")
  with gr.Tab("Rcord Meeting"):  
    gr.Markdown(value=f"## Emp Name: {EMPLOYEE_NAME}")
    with gr.Row():
      gr.Markdown(f"Emp ID: {EMPLOYEE_ID}")
      gr.Markdown(f"Branch: {EMPLOYEE_BRANCH}")
    gr.Markdown("---")
    with gr.Row():
      with gr.Column():
        customer_id = gr.Textbox(value=CUSTOMER_ID,
                                 label="Customer ID",
                                 type="text", 
                                 interactive=True)
        customer_name = gr.Textbox(value=CUSTOMER_NAME, 
                                   label="Customer Name",
                                   type="text", 
                                   interactive=True)
      with gr.Column():
        meeting_date = gr.Textbox(value=DATE,
                                  label="Meeting Date", 
                                  type="text",
                                  interactive=True)
        meeting_time = gr.Textbox(value=TIME, 
                                  label="Meeting Time", 
                                  type="text", 
                                  interactive=True)
    with gr.Row(): 
      with gr.Box():
        with gr.Row(): 
          audio_record = gr.Microphone(type="filepath",
                                      label="1. Meeting Summary"
                                      )
          audio_clear_button = gr.Button("Clear Recording")
        instruction = gr.Textbox(value=INSTRUCTION, 
                                 label="Recording Instructions")
      with gr.Box():
        transcription = gr.Textbox(label="Transcription", 
                                  type="text", 
                                  interactive=True,
                                  placeholder = "Audio Transcription will appear here", 
                                  max_lines=10)
    with gr.Row():
      short_term_goals = gr.Textbox(label="Short term goals",
                                    type = "text",
                                    interactive = True,
                                    placeholder = "Short term goals of the client",
                                    max_lines = 5
                                    )
      long_term_goals = gr.Textbox(label="Long term goals",
                                    type = "text",
                                    interactive = True,
                                    placeholder = "Long term goals of the client",
                                    max_lines = 5
                                    )
    with gr.Row():
      new_meeting_button = gr.Button("Record New Meeting")
      rec_products = gr.Button("2. Recommend HSBC Solutions")
    with gr.Row():
      short_term_rec = gr.Label(label="Short-term recommendation")
      long_term_rec = gr.Label(label="Long-term recommendation")

    with gr.Box():
      gr.Markdown("## RM recommended solutions")
      with gr.Row():
        rm_rec_sort = gr.Dropdown(choices=products,
                                  multiselect = True,
                                  label="Short Term",
                                  info="RM recommended solutions, suitable for the short-term goal")
        rm_rec_long = gr.Dropdown(choices=products,
                                  multiselect=True,
                                  label="Long Term",
                                  info="RM recommended solutions suitable for the long-term goal")
    submit_button = gr.Button("4. Submit Record")
      
  with gr.Tab("Data"):
    rm_df = gr.State(pd.DataFrame())
    rec_df = gr.State(pd.DataFrame())
    gr.Markdown("## Meeting Information Data")
    df_out = gr.Dataframe(wrap=True)
    gr.Markdown("---")
    gr.Markdown("## Recommendation Data")
    df_rec_out = gr.Dataframe(wrap=True)
  
  
  audio_record.change(update_transcription_text,audio_record,transcription)
  audio_clear_button.click(clear_audio_button,[],audio_record)
  transcription.change(update_short_long_term_goals,
                       transcription,
                       [short_term_goals, 
                        long_term_goals]
                       )
  new_meeting_button.click(record_new_meeting,[],[audio_record, 
                                                  transcription
                                                  ])
  rec_products.click(recommend_hsbc_product, 
                     [short_term_goals,long_term_goals],
                     [short_term_rec,long_term_rec]
                     )
  submit_button.click(submit_all_records,
                      [rm_df, rec_df, customer_id,customer_name,meeting_date,meeting_time,
                       transcription, short_term_rec,long_term_rec,
                       rm_rec_sort,rm_rec_long],
                      [rm_df, df_out, rec_df, df_rec_out])

## launching the app

In [ ]:
demo.launch(share=True,debug=False)